# Forecasting, hierarchical data, tuning, and more

In this notebook, we will cover more advanced forecasting topics, specially focused on hierarchical data, tuning, and reconciliation.
We will use sales data from [this kaggle dataset](https://www.kaggle.com/datasets/utathya/future-volume-prediction?resource=download), which contains sales data for different products (SKUs) and agencies.

The notebook will be divided into the following sections:

1. Data preparation for hierarchical forecasting
2. Simple forecasting with builtin parallelization
3. Tuning with Optuna
4. Tuning indivually for each timeseries
5. Reconciliation
6. Benchmarking


In [1]:
import warnings
import logging

warnings.filterwarnings("ignore")
logger = logging.getLogger('cmdstanpy')
logger.setLevel(logging.ERROR)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Loading and preparing the data

The dataset is a 3-level hierarchical time series, with the following levels:

1. Total sales for all SKUs and agencies
2. Sales for each agency
3. Sales for each SKU in each agency


```mermaid
graph TD
    Root["__total"] --> Agency_01
    Root --> Agency_02
    Root --> Agency_60
    
    Agency_01 --> SKU_01_A01["SKU_01"]
    Agency_01 --> SKU_02_A01["SKU_02"]
    Agency_01 --> SKU_11_A01["SKU_11"]
    Agency_01 --> Agency_01_Total["__total"]
    
    Agency_02 --> SKU_01_A02["SKU_01"]
    Agency_02 --> SKU_02_A02["SKU_02"]
    Agency_02 --> SKU_03_A02["SKU_03"]
    Agency_02 --> Agency_02_Total["__total"]
    
    Agency_60 --> SKU_01_A60["SKU_01"]
    Agency_60 --> SKU_02_A60["SKU_02"]
    Agency_60 --> SKU_23_A60["SKU_23"]
    Agency_60 --> Agency_60_Total["__total"]

```

In sktime, we use pandas multiindex to represent the hierarchy, where each level in the index represent a level in the hierarchy. The last level is reserved to the time index.

In [3]:
# TODO: put in a function
dataset = pd.read_csv("data/stallion_data.csv")
dataset["date"] = pd.to_datetime(dataset["date"], format="%Y-%m-01").dt.to_period("M")
y  = dataset.set_index(["agency", "sku", "date"])[["volume"]].sort_index()
y

volume
agency    sku    date            
Agency_01 SKU_01 2013-01   80.676
                 2013-02   98.064
                 2013-03  133.704
                 2013-04  147.312
                 2013-05  175.608
...                           ...
Agency_60 SKU_23 2017-08    1.980
                 2017-09    1.260
                 2017-10    0.990
                 2017-11    0.090
                 2017-12    2.250

[21000 rows x 1 columns]

## 1.1 Some useful pandas multiindex operations

Multiindex is a powerful tool in pandas, and knowing its operations can be very useful when working with hierarchical data.


* `df.index.get_level_values(level)`: returns the values of a specific level in the multiindex.
* `df.index.droplevel(level)`: drops a specific level from the multiindex.
* `df.index.loc["value"]`: select rows whose index level 0 is "value".
* `df.index.loc[pd.IndexSlice[:, "value"], :]`: select rows whose index level 1 is "value".

In sktime, for example, one can use `df.index.droplevel(-1).unique()` to get the timeseries in the dataset

In [4]:
y.index.get_level_values(0)

Index(['Agency_01', 'Agency_01', 'Agency_01', 'Agency_01', 'Agency_01',
       'Agency_01', 'Agency_01', 'Agency_01', 'Agency_01', 'Agency_01',
       ...
       'Agency_60', 'Agency_60', 'Agency_60', 'Agency_60', 'Agency_60',
       'Agency_60', 'Agency_60', 'Agency_60', 'Agency_60', 'Agency_60'],
      dtype='object', name='agency', length=21000)

In [5]:
y.index.get_level_values(-1)

PeriodIndex(['2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
             '2013-07', '2013-08', '2013-09', '2013-10',
             ...
             '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08',
             '2017-09', '2017-10', '2017-11', '2017-12'],
            dtype='period[M]', name='date', length=21000)

In [6]:
y.index.droplevel(-1).unique()

MultiIndex([('Agency_01', 'SKU_01'),
            ('Agency_01', 'SKU_02'),
            ('Agency_01', 'SKU_03'),
            ('Agency_01', 'SKU_04'),
            ('Agency_01', 'SKU_05'),
            ('Agency_01', 'SKU_11'),
            ('Agency_02', 'SKU_01'),
            ('Agency_02', 'SKU_02'),
            ('Agency_02', 'SKU_03'),
            ('Agency_02', 'SKU_04'),
            ...
            ('Agency_59', 'SKU_05'),
            ('Agency_59', 'SKU_07'),
            ('Agency_59', 'SKU_17'),
            ('Agency_60', 'SKU_01'),
            ('Agency_60', 'SKU_02'),
            ('Agency_60', 'SKU_03'),
            ('Agency_60', 'SKU_04'),
            ('Agency_60', 'SKU_05'),
            ('Agency_60', 'SKU_07'),
            ('Agency_60', 'SKU_23')],
           names=['agency', 'sku'], length=350)

## 1.2 Aggregating and visualizing the data

Since the dataset do not come with totals for each level, we will need to add them.
It can be easily done with `Aggregator` transformer from sktime.

In [7]:
from sktime.transformations.hierarchical.aggregate import Aggregator

y = Aggregator().fit_transform(y)
y

volume
agency    sku     date                  
Agency_01 SKU_01  2013-01      80.676000
                  2013-02      98.064000
                  2013-03     133.704000
                  2013-04     147.312000
                  2013-05     175.608000
...                                  ...
__total   __total 2017-08  599553.665250
                  2017-09  556966.701300
                  2017-10  542554.007475
                  2017-11  457914.412950
                  2017-12  553316.088000

[24540 rows x 1 columns]

### Train-test split

In [8]:
from sktime.forecasting.model_selection import temporal_train_test_split
y_train, y_test = temporal_train_test_split(y, test_size=18)

test_timeindex = y_test.index.get_level_values(-1).unique()
test_timeindex

PeriodIndex(['2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12',
             '2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06',
             '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12'],
            dtype='period[M]', name='date')

In [9]:
from utils import display_hierarchical_timeseries

display_hierarchical_timeseries(y_train, y_test)

interactive(children=(Dropdown(description='Level 0:', options=('Agency_01', 'Agency_02', 'Agency_03', 'Agency…

## 2. Parallelization

Instead of needing to manually iterate over the series, we can use the builtin parallelization to handle this 🙂.

When a univariate forecasting model is fitted to a hierarchical time series, one model copy is created for each series in the hierarchy and fitted separately. All models share the same hyperparameter.

In [14]:
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.fbprophet import Prophet

logger = logging.getLogger('cmdstanpy')
logger.setLevel(logging.ERROR)


In [15]:

model = Prophet(freq="Q")
model.fit(y_train)

test_predictions = model.predict(fh=test_timeindex)
test_predictions


volume
agency    sku     date                  
Agency_01 SKU_01  2016-07      34.567985
                  2016-08      82.860599
                  2016-09      62.245607
                  2016-10      76.752518
                  2016-11      13.102124
...                                  ...
__total   __total 2017-08  549435.017496
                  2017-09  485786.520166
                  2017-10  522701.951932
                  2017-11  474187.879867
                  2017-12  606214.921197

[7362 rows x 1 columns]

In [16]:
model.forecasters_

forecasters
Agency_01 SKU_01   Prophet(freq='Q')
          SKU_02   Prophet(freq='Q')
          SKU_03   Prophet(freq='Q')
          SKU_04   Prophet(freq='Q')
          SKU_05   Prophet(freq='Q')
...                              ...
Agency_60 SKU_05   Prophet(freq='Q')
          SKU_07   Prophet(freq='Q')
          SKU_23   Prophet(freq='Q')
          __total  Prophet(freq='Q')
__total   __total  Prophet(freq='Q')

[409 rows x 1 columns]

## 3. Tuning hyperparameters with Optuna

Optuna is a hyperparameter optimization framework that supports many sampling strategies. The default is Tree of Parzen Estimators (TPE), which is a Bayesian-like optimization algorithm.

We need to define the search space, which may vary depending on the nature of hyperparemeter.
Below, we tune some hyperparameters for demonstration purposes, with few evaluations to speed up the process.

In [17]:
from sktime.forecasting.model_selection import (ForecastingOptunaSearchCV)
from sktime.split import ExpandingWindowSplitter
from optuna.distributions import CategoricalDistribution, IntUniformDistribution, LogUniformDistribution

cv = ExpandingWindowSplitter(fh=[0,1,2,3], initial_window=36, step_length=12)

tuning_model = ForecastingOptunaSearchCV(
    forecaster=Prophet(freq="Q"),
    param_grid={"n_changepoints":IntUniformDistribution(2,20),
                "yearly_seasonality":CategoricalDistribution([True, False]),
                "seasonality_mode":CategoricalDistribution(["additive", "multiplicative"]),
                "changepoint_prior_scale":LogUniformDistribution(0.0001, 0.01),
                "seasonality_prior_scale":LogUniformDistribution(0.0001, 10),},
    cv=cv,
    n_evals=2
)

tuning_model.fit(y_train)



[I 2024-08-15 22:41:16,960] A new study created in memory with name: no-name-d456440c-1a80-44c7-a5f1-cea478bc790d


ForecastingOptunaSearchCV(cv=ExpandingWindowSplitter(fh=[0, 1, 2, 3],
                                                     initial_window=36,
                                                     step_length=12),
                          forecaster=Prophet(freq='Q'), n_evals=2,
                          param_grid={'changepoint_prior_scale': LogUniformDistribution(high=0.01, low=0.0001),
                                      'n_changepoints': IntUniformDistribution(high=20, low=2, step=1),
                                      'seasonality_mode': CategoricalDistribution(choices=('additive', 'multiplicative')),
                                      'seasonality_prior_scale': LogUniformDistribution(high=10.0, low=0.0001),
                                      'yearly_seasonality': CategoricalDistribution(choices=(True, False))})

In [18]:
tuning_model.best_params_

{'n_changepoints': 9,
 'yearly_seasonality': True,
 'seasonality_mode': 'additive',
 'changepoint_prior_scale': 0.00040657067682418245,
 'seasonality_prior_scale': 0.07595980655771152}

In [19]:
tuning_model.best_forecaster_

Prophet(changepoint_prior_scale=0.00040657067682418245, freq='Q',
        n_changepoints=9, seasonality_prior_scale=0.07595980655771152,
        yearly_seasonality=True)

## 4. Tuning each series individually

In the example above, we tuned the hyperparameter that performs the best on average for all timeseries.
However, it is possible that the best hyperparameter for each series is different. 

We will use `ForecastBylevel` to apply tuning separately for each series in the hierarchy. 

In [20]:
from sktime.forecasting.compose import ForecastByLevel


tune_by_level = ForecastByLevel(
    forecaster=tuning_model.set_params(n_evals=1),
    groupby="local"
)

tune_by_level.fit(y_train)

tuned_by_level_predictions = tune_by_level.predict(fh=test_timeindex)

[I 2024-08-15 22:45:30,001] A new study created in memory with name: no-name-d382c594-7c4c-43d0-8252-674ba0265e49
[I 2024-08-15 22:45:30,239] A new study created in memory with name: no-name-496e6c1a-d75d-4add-881f-45be3e42a01e
[I 2024-08-15 22:45:30,489] A new study created in memory with name: no-name-d0b8c091-dfb2-4a1f-9b8f-d76a1a36e49d
[I 2024-08-15 22:45:30,873] A new study created in memory with name: no-name-af1c6a34-8233-4e25-b6bf-f93084a112dd
[I 2024-08-15 22:45:31,180] A new study created in memory with name: no-name-a9bbad26-e6c8-41a0-bf9d-15b4b7d90f47
[I 2024-08-15 22:45:32,695] A new study created in memory with name: no-name-0266b7d1-a379-46ce-b1e9-45aaab9ac73f
[I 2024-08-15 22:45:32,962] A new study created in memory with name: no-name-929b6b88-2d50-4fc6-8a04-29effc8e4800
[I 2024-08-15 22:45:33,796] A new study created in memory with name: no-name-793bc061-7b80-434c-be73-d1e6d3e85911
[I 2024-08-15 22:45:34,313] A new study created in memory with name: no-name-2dd74b6c-e0

In [21]:
tune_by_level.forecasters_

forecasters
Agency_01 SKU_01   ForecastByLevel(forecaster=ForecastingOptunaSe...
          SKU_02   ForecastByLevel(forecaster=ForecastingOptunaSe...
          SKU_03   ForecastByLevel(forecaster=ForecastingOptunaSe...
          SKU_04   ForecastByLevel(forecaster=ForecastingOptunaSe...
          SKU_05   ForecastByLevel(forecaster=ForecastingOptunaSe...
...                                                              ...
Agency_60 SKU_05   ForecastByLevel(forecaster=ForecastingOptunaSe...
          SKU_07   ForecastByLevel(forecaster=ForecastingOptunaSe...
          SKU_23   ForecastByLevel(forecaster=ForecastingOptunaSe...
          __total  ForecastByLevel(forecaster=ForecastingOptunaSe...
__total   __total  ForecastByLevel(forecaster=ForecastingOptunaSe...

[409 rows x 1 columns]

In [22]:
tune_by_level.forecasters_.forecasters.apply(lambda x: x.forecaster_.best_params_).to_frame("Best params")

Best params
Agency_01 SKU_01   {'n_changepoints': 14, 'yearly_seasonality': F...
          SKU_02   {'n_changepoints': 7, 'yearly_seasonality': Tr...
          SKU_03   {'n_changepoints': 14, 'yearly_seasonality': T...
          SKU_04   {'n_changepoints': 12, 'yearly_seasonality': T...
          SKU_05   {'n_changepoints': 15, 'yearly_seasonality': T...
...                                                              ...
Agency_60 SKU_05   {'n_changepoints': 18, 'yearly_seasonality': F...
          SKU_07   {'n_changepoints': 12, 'yearly_seasonality': F...
          SKU_23   {'n_changepoints': 16, 'yearly_seasonality': T...
          __total  {'n_changepoints': 16, 'yearly_seasonality': F...
__total   __total  {'n_changepoints': 17, 'yearly_seasonality': T...

[409 rows x 1 columns]

## 5. Reconciliation

Probably, your forecasts won't be _coherent_ with respect to the hierarchy. The sum of the forecasts for each series in a level will not be equal to the forecast for the total of that level.

This can mean two things:

1. By definition, one of them is wrong.
2. The users of the forecasts will not be happy.

In [23]:
def check_unconsistency(preds):
    total_level_predictions = preds.loc[("__total", "__total")]
    total_from_bottom_level_predictions = preds.loc[ (preds.index.get_level_values(1) != "__total")].groupby(level=-1).sum()

    difference = total_level_predictions - total_from_bottom_level_predictions
    return difference / total_from_bottom_level_predictions

check_unconsistency(tuned_by_level_predictions)

,volume
date,
2016-07,-0.001286
2016-08,0.001100
2016-09,-0.045759
2016-10,-0.048599
2016-11,-0.083877
2016-12,0.084144
2017-01,-0.135921
2017-02,-0.086797
2017-03,0.036871


There are, fortunately, techniques to fix this. We call them `reconciliation` techniques.

The hierarchy constrains, such as the sum of children must be equal to the parent,
are a set of linear constraints, and we have some strategies to satisfy them:

1. Bottom-up (`bu`): we forecast the series at the lowest level, and then we aggregate them to the higher levels.
2. Forecast Proportions (`td_fcst`): we use the forecasts at bottom levels to estimate the proportions with respect to the total, and then we multiply them by the total forecast
3. Orthogonal Projection (`ols`): a.k.a. ordinary least squares, this amounts to using the linear contraints to build a projection matrix that takes the forecasts to a hyperplane that satisfies the constraints.
4. Oblique Projection (`wls_str`): this performs a weighted least squares projection, considering the scale of the series before computing the projection.
4. Mint Shrink (`mint`): this is a more advanced technique that uses the information in the training set to estimate the covariance matrix of the errors, and then it uses this information choose the oblique projection that minimizes the mean squared error of the reconciled forecasts.


In [24]:
from sktime.transformations.hierarchical.reconcile import Reconciler

reconciler = Reconciler(method="ols") # mint, bu, td_fcst
reconciled_predictions = reconciler.fit_transform(tuned_by_level_predictions)

In [25]:
check_unconsistency(reconciled_predictions)

,volume
date,
2016-07,0.000000e+00
2016-08,1.080667e-15
2016-09,4.702497e-16
2016-10,0.000000e+00
2016-11,-7.466720e-16
2016-12,-1.926531e-16
2017-01,4.019068e-16
2017-02,-4.905003e-16
2017-03,-2.072404e-16


### Using pipelines to reconcile

In [26]:
from sktime.forecasting.compose import TransformedTargetForecaster

model_with_reconciler = TransformedTargetForecaster(
    steps=[
        ("forecaster", model),
        ("reconciler", Reconciler(method="ols"))
    ]
)

model_with_reconciler.fit(y_train)
reconciled_predictions = model_with_reconciler.predict(fh=test_timeindex)

In [27]:
check_unconsistency(reconciled_predictions)

,volume
date,
2016-07,0.000000e+00
2016-08,8.514605e-16
2016-09,-2.529464e-16
2016-10,-2.193840e-16
2016-11,3.735398e-16
2016-12,-1.959765e-16
2017-01,7.828664e-16
2017-02,4.753298e-16
2017-03,4.169450e-16


In [28]:
from sktime.transformations.hierarchical.reconcile import Reconciler


model_with_reconciler = model * Reconciler(method="ols")

model_with_reconciler.fit(y_train)
reconciled_predictions = model_with_reconciler.predict(fh=test_timeindex)

In [29]:
from sktime.performance_metrics.forecasting import MeanSquaredScaledError

metric = MeanSquaredScaledError(multilevel="raw_values")
metric(y_train=y_train, y_true=y_test, y_pred=test_predictions.loc[y_test.index])

MeanSquaredScaledError
Agency_01 SKU_01                 1.952239
          SKU_02                 2.587613
          SKU_03                 1.364352
          SKU_04                 3.852825
          SKU_05                 0.617581
...                                   ...
Agency_60 SKU_05                 1.405386
          SKU_07                 2.142524
          SKU_23                 5.305738
          __total                0.548579
__total   __total                0.258232

[409 rows x 1 columns]

## 6. Benchmarking

In [30]:
metric = MeanSquaredScaledError(multilevel="uniform_average",)
metric(y_train=y_train, y_true=y_test, y_pred=test_predictions.loc[y_test.index])

6.718007298385789e+16

In [31]:
from sktime.performance_metrics.forecasting import MeanSquaredScaledError

metric = MeanSquaredScaledError(multilevel="uniform_average_time")
(metric(y_train=y_train, y_true=y_test, y_pred=tuned_by_level_predictions.loc[y_test.index]),
 metric(y_train=y_train, y_true=y_test, y_pred=Reconciler(method="ols").fit_transform(tuned_by_level_predictions.loc[y_test.index])))

(0.1601289332698118, 0.15684205061995155)

In [32]:
metric = MeanSquaredScaledError(multilevel="uniform_average_time")
metric(y_train=y_train, y_true=y_test, y_pred=test_predictions.loc[y_test.index])

0.19981150402265496